In [1]:
import numpy as np
import pandas as pd
import os
import nltk
import pickle
import re

from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util

c:\Users\poomk\miniconda3\envs\dataenv\lib\site-packages\huggingface_hub\snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [2]:
# get the cleaned songs data
songs_set = pd.read_csv('../songsdata/cleaned_data/cleaned_songs.csv')

In [3]:
# get songs lyrics set
lyrics_set = songs_set['lyrics']

In [4]:
# pickle the sets
l_pickle = [songs_set, lyrics_set]
with open('../pickle_objects/song_lyrics_set.obj', 'wb') as f:
    pickle.dump(l_pickle, f)

In [5]:
# get embedding function
def embeddings_gen(lyrics_set, model_name = "all-distilroberta-v1"):
    # Create mdoel
    model = SentenceTransformer(model_name)
    # Create bag of lyrics lines with their corresponding song_ids
    l_lyrics_lines =[]
    l_song_idx =[]
    for idx in lyrics_set.index:
        lyrics = lyrics_set[idx]
        lyrics_lines = re.split('\n',lyrics)
        # Condition: do not include lyrics lines that are more than 512 tokens
        if any(len(word_tokenize(x)) >= 512 for x in lyrics_lines):
            continue
        l_lyrics_lines.extend(lyrics_lines)
        l_song_idx.extend([idx] * len(lyrics_lines))

    # For invert indexing //Store related song ids as np array
    arr_song_idx = np.array(l_song_idx)
    # Store lyrics lines as np array
    arr_lyrics_idx = np.array(l_lyrics_lines)

    embeddings = model.encode(l_lyrics_lines, convert_to_tensor=True)
    
    return embeddings, arr_song_idx, arr_lyrics_idx


In [6]:
with open('../pickle_objects/song_lyrics_set.obj', 'rb') as f:
    l_pickle = pickle.load(f)

lyrics_set = l_pickle[1]

In [7]:
embeddings, arr_song_idx, arr_lyrics_idx = embeddings_gen(lyrics_set)

KeyboardInterrupt: 

In [ ]:
l_pickle = [embeddings, arr_song_idx, arr_lyrics_idx]
with open('../pickle_objects/embeddings_indices.obj', 'wb') as f:
    pickle.dump(l_pickle, f)